###  Contextualização

Uma Fábrica de celulose tem 3 fazendas com materia prima(madeira) disponível para coleta e 3 empresas de logistica aptas a realizarem o transporte. Os custos unitários de transporte de cada fazenda para cada transportadora, assim como a quantidade de cargas disponíveis em cada fazenda e a quantidade de veículos por transportadora encontram-se na tabela abaixo. 
O objetivo é coletar a madeira em campo, respeitando a quantidade de madeira disponível, de forma a minimizar o custo total de transporte. Modelar o problema de transporte.

| Origem | Transportadora 1 | Transportadora 2 | Transportadora 3 |  Disponível |
| --- | --- | --- | --- | --- | 
| Fazenda 1 | 9.8 | 10.1 | 9.9 | **100** |
| Fazenda 2| 12.1 | 11.9 | 12.3 | **140** |
| Fazenda 3 | 8.5 | 8.8 | 8.3 | **160** |
| Capacidade | **120** | **130** | **150** | 


### Modelo Genérico

**Índices/Conjuntos**

**I**: Conjunto de Transportadoras {1,2,...,m}

**J** : Conjunto de fazendas {1,2,...,n}

**Parâmetros**

$ c_{ij}$: Custo unitário de transporte pela Transportadora $i \in I $ a partir da fazenda $j \in J$

$a_{i}$: Quantidade de veículos disponíveis pela transportadora $ i \in I$

$b_{j}$: Quantidade de cargas para retirada da fazenda $ j \in J$

 **Variáveis de decisão**

$x_{ij}$: Cargas transportadas pela transportadora $ i \in I$ a partir da fazenda $j \in J$ - O quanto será transportado de cada fazenda por cada transportadora

### Formulação matemática

$ min z = \sum\limits_{i\in I} \sum\limits_{j \in J} c_{ij} x_{ij}$

Onde:

$\sum\limits_{j \in J} x_{ij} \leq a_{i}, \forall \; i \in I$ - garante que a transportadora não vai enviar mais veículos do que possui

$\sum\limits_{i \in I} x_{ij} = b_{j}, \forall \; j \in J$ - garante que toda a madeira seja retirada

$x_{ij} \geq 0 \;\; \forall i \in I \text{, }j \in J$

Obs.: A principal vantagem de utilizar o modelo genérico é que através dele podemos utilizar loops de preenchimento dos nossos dados reais no pyomo de acordo com a estrutura da nossa formulação. Essa prática é extremamente necessária em problemas maiores, caso contrário precisaríamos inserir as restrições uma por uma. 

### Preparando os dados

In [1]:
custos = [[9.8,10.1,9.9],
          [12.1,11.9,12.3],
          [8.5,8.8,8.3]]

veiculos = [120,130,150]

cargas = [100,140,160]

In [2]:
m = len(veiculos)
n = len(cargas)

### Modelo computacional

In [3]:
import pyomo.environ as pyo

In [4]:
#Instanciando o modelo:

modelo = pyo.ConcreteModel()

In [5]:
# Declaração dos conjuntos:

modelo.I = pyo.RangeSet(m)
modelo.J = pyo.RangeSet(n)

In [6]:
modelo.pprint()

2 RangeSet Declarations
    I : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]
    J : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]

2 Declarations: I J


In [7]:
modelo.c = pyo.Param(modelo.I, modelo.J, initialize=lambda modelo, i, j: custos[i-1][j-1])

In [8]:
modelo.pprint()

1 Set Declarations
    c_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)}

2 RangeSet Declarations
    I : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]
    J : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]

1 Param Declarations
    c : Size=9, Index=c_index, Domain=Any, Default=None, Mutable=False
        Key    : Value
        (1, 1) :   9.8
        (1, 2) :  10.1
        (1, 3) :   9.9
        (2, 1) :  12.1
        (2, 2) :  11.9
        (2, 3) :  12.3
        (3, 1) :   8.5
        (3, 2) :   8.8
        (3, 3) :   8.3

4 Declarations: I J c_index c


In [9]:
modelo.a = pyo.Param(modelo.I, initialize=lambda modelo, i: veiculos[i-1])

In [10]:
modelo.b = pyo.Param(modelo.J, initialize=lambda modelo, j: cargas[j-1])

In [11]:
modelo.pprint()

1 Set Declarations
    c_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)}

2 RangeSet Declarations
    I : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]
    J : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]

3 Param Declarations
    a : Size=3, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   120
          2 :   130
          3 :   150
    b : Size=3, Index=J, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   100
          2 :   140
          3 :   160
    c : Size=9, Index=c_index, Domain=Any, Default=None, Mutable=False
        Key    : Value
        (1, 1) :   9.8
        (1, 2) :  10.1
        (1, 3) :   9.9
        (2, 1) :  12.1
        (2, 2) :  11.9
        (2, 3) :  12.3


In [12]:
modelo.x = pyo.Var(modelo.I, modelo.J, within=pyo.NonNegativeReals)

In [13]:
modelo.pprint()

2 Set Declarations
    c_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)}

2 RangeSet Declarations
    I : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]
    J : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]

3 Param Declarations
    a : Size=3, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   120
          2 :   130
          3 :   150
    b : Size=3, Index=J, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   100
          2 :   140
          3 :   160
    c : Size=9, Index=c_in

#### Função objetivo:

Conforme já pontuamos, o intuito é garantir o menor custo possível na atividade, logo:

$ min z = \sum\limits_{i\in I} \sum\limits_{j \in J} c_{ij} x_{ij}$

In [14]:
# criar uma função auxiliar:

def regra_transp(mod):
    return pyo.summation(mod.c, mod.x) #summation faz a soma dos produtos

In [15]:
# agora vamos incluir a função criada no pyomo:
modelo.z = pyo.Objective(rule=regra_transp, sense=pyo.minimize)

$\sum\limits_{j \in J} x_{ij} \leq a_{i}, \forall \; i \in I$ - garante que a transportadora não vai enviar mais veículos do que possui

In [16]:
# Declaração das restrições:

def regra_restr_veic(mod, i):
    return sum(mod.x[i,j] for j in mod.J) <= mod.a[i]
    
modelo.restr_cap = pyo.Constraint(modelo.I, rule=regra_restr_veic)

$\sum\limits_{i \in I} x_{ij} = b_{j}, \forall \; j \in J$ - Quantidade de cargas por fazenda

In [18]:
def regra_cargas(mod, j):
    return sum(mod.x[i,j] for i in mod.I) == mod.b[j]

modelo.restr_disp = pyo.Constraint(modelo.J, rule=regra_cargas)

In [19]:
import subprocess

def patched_subprocess_run(*args, **kwargs):
    if kwargs.get("timeout") is not None:
        kwargs["timeout"] = 7
    return orig_subprocess_run(*args, **kwargs)

orig_subprocess_run = subprocess.run
subprocess.run = patched_subprocess_run

result = pyo.SolverFactory('glpk').solve(modelo)

In [20]:
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 3972.0
  Upper bound: 3972.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 10
  Number of nonzeros: 19
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.06993556022644043
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

In [21]:
modelo.x.pprint()

x : Size=9, Index=x_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 : 100.0 :  None : False : False : NonNegativeReals
    (1, 2) :     0 :  10.0 :  None : False : False : NonNegativeReals
    (1, 3) :     0 :  10.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 2) :     0 : 130.0 :  None : False : False : NonNegativeReals
    (2, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (3, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (3, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (3, 3) :     0 : 150.0 :  None : False : False : NonNegativeReals


In [22]:
modelo.x[1,1]()

100.0

In [23]:
for i in modelo.I:
    for j in modelo.J:
        print(f'da Transporadora {i} para a fazenda {j}, enviar {modelo.x[i,j]()} caminhões')

da Transporadora 1 para a fazenda 1, enviar 100.0 caminhões
da Transporadora 1 para a fazenda 2, enviar 10.0 caminhões
da Transporadora 1 para a fazenda 3, enviar 10.0 caminhões
da Transporadora 2 para a fazenda 1, enviar 0.0 caminhões
da Transporadora 2 para a fazenda 2, enviar 130.0 caminhões
da Transporadora 2 para a fazenda 3, enviar 0.0 caminhões
da Transporadora 3 para a fazenda 1, enviar 0.0 caminhões
da Transporadora 3 para a fazenda 2, enviar 0.0 caminhões
da Transporadora 3 para a fazenda 3, enviar 150.0 caminhões


In [24]:
modelo.pprint()

2 Set Declarations
    c_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)}

2 RangeSet Declarations
    I : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]
    J : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]

3 Param Declarations
    a : Size=3, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   120
          2 :   130
          3 :   150
    b : Size=3, Index=J, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   100
          2 :   140
          3 :   160
    c : Size=9, Index=c_in